In [ ]:
!pip install rioxarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 34.1 MB/s eta 0:00:00


In [ ]:
!pip install zarr complete

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pyproj
import pandas as pd
import xarray as xr
import geopandas as gpd
import rioxarray

In [ ]:
gdf = gpd.read_file('/content/drive/Shareddrives/CalFire Biomass LCA/Technical Work/CBREC-LCA_timber_activity_polygons_2016-2019')

In [ ]:
gdf

,year,treat_code,acre,id,OBJECTID,geometry
0,2016,Clearcut,2.314122,exempt,1,"MULTIPOLYGON (((-336757.406 311378.573, -33674..."
1,2016,Clearcut,1.310398,exempt,4,"MULTIPOLYGON (((-274866.743 273132.394, -27486..."
2,2016,Clearcut,1.868513,exempt,6,"POLYGON ((-274555.272 272881.305, -274574.179 ..."
3,2016,Clearcut,2.564922,exempt,9,"POLYGON ((-274422.223 272302.237, -274365.875 ..."
4,2016,20_Proportional_Thin,22.974324,exempt,12,"POLYGON ((-319429.284 231362.308, -319441.259 ..."
...,...,...,...,...,...,...
13022,2019,Clearcut,19.919865,thp,51427,"POLYGON ((-152348.476 352182.458, -152355.592 ..."
13023,2019,Clearcut,9.069485,thp,51428,"POLYGON ((-152874.441 351520.403, -152894.414 ..."
13024,2019,60_Proportional_Thin,0.358057,thp,51429,"POLYGON ((-337076.701 394573.734, -337098.286 ..."
13025,2019,60_Proportional_Thin,5.415165,thp,51430,"POLYGON ((-336991.188 395094.546, -337004.303 ..."


In [ ]:
polygon_6 = xr.open_zarr('/content/drive/Shareddrives/CalFire Biomass LCA/Technical Work/CollectionProcessing/tiny2')


In [ ]:
def polygon_fetcher(id):
    try:
        poly = gdf.loc[gdf.OBJECTID == id].iloc[0].geometry
        if poly.geom_type == 'MultiPolygon':
            return list(poly.geoms)
        elif poly.geom_type == 'Polygon':
            return [poly]
    except:
        print("Error with polygon "+str(id))
        return 1

In [ ]:
polygon_fetcher(6)

[<POLYGON ((-274555.272 272881.305, -274574.179 272927.777, -274466.67 272968...>]

In [ ]:
list(polygon_6)

['CWD_cm',
 'Cell_To_Road',
 'FCID',
 'FWD_cm',
 'Foliage_cm',
 'Landscape',
 'Merchantable',
 'Nearest_Plant',
 'Nearest_Plant_ID',
 'Recovered',
 'Slope']

In [ ]:
proj4 = '+proj=aea +lat_0=0 +lon_0=-120 +lat_1=34 +lat_2=40.5 +x_0=0 +y_0=-4000000 +ellps=GRS80 +units=m +no_defs '
crs = pyproj.CRS.from_proj4(proj4)

In [ ]:
polygon_6

<xarray.Dataset>
Dimensions:           (x: 7, y: 5, treatment: 13, case: 54, disposition: 2,
                       type: 5, size_class: 3)
Coordinates:
  * x                 (x) float64 -2.746e+05 -2.746e+05 ... -2.744e+05
  * y                 (y) float64 2.729e+05 2.729e+05 ... 2.729e+05 2.73e+05
Dimensions without coordinates: treatment, case, disposition, type, size_class
Data variables:
    CWD_cm            (x, y) float16 dask.array<chunksize=(7, 5), meta=np.ndarray>
    Cell_To_Road      (x, y) float16 dask.array<chunksize=(7, 5), meta=np.ndarray>
    FCID              (x, y) float64 dask.array<chunksize=(7, 5), meta=np.ndarray>
    FWD_cm            (x, y) float16 dask.array<chunksize=(7, 5), meta=np.ndarray>
    Foliage_cm        (x, y) float16 dask.array<chunksize=(7, 5), meta=np.ndarray>
    Landscape         (x, y, treatment, case, disposition, type) float16 dask.array<chunksize=(7, 5, 13, 54, 2, 5), meta=np.ndarray>
    Merchantable      (x, y, treatment, case) float16 dask.array<chunksize=(7, 5, 13, 54), meta=np.ndarray>
    Nearest_Plant     (x, y) float16 dask.array<chunksize=(7, 5), meta=np.ndarray>
    Nearest_Plant_ID  (x, y) float16 dask.array<chunksize=(7, 5), meta=np.ndarray>
    Recovered         (x, y, treatment, case, size_class) float16 dask.array<chunksize=(7, 5, 13, 54, 3), meta=np.ndarray>
    Slope             (x, y) float16 dask.array<chunksize=(7, 5), meta=np.ndarray>

In [ ]:
        masked = polygon_6[['Cell_To_Road','Nearest_Plant','Nearest_Plant_ID','Recovered','Slope']]
        df_temp = masked.to_dataframe()

In [ ]:
df_temp

Cell_To_Road  \
x              y             treatment case size_class                 
-274600.584639 272853.439868 0         0    0                   60.0   
                                            1                   60.0   
                                            2                   60.0   
                                       1    0                   60.0   
                                            1                   60.0   
...                                                              ...   
-274420.584639 272973.439868 12        52   1                   84.0   
                                            2                   84.0   
                                       53   0                   84.0   
                                            1                   84.0   
                                            2                   84.0   

                                                        Nearest_Plant  \
x              y             treatment case size_class                  
-274600.584639 272853.439868 0         0    0                    75.0   
                                            1                    75.0   
                                            2                    75.0   
                                       1    0                    75.0   
                                            1                    75.0   
...                                                               ...   
-274420.584639 272973.439868 12        52   1                    75.0   
                                            2                    75.0   
                                       53   0                    75.0   
                                            1                    75.0   
                                            2                    75.0   

                                                        Nearest_Plant_ID  \
x              y             treatment case size_class                     
-274600.584639 272853.439868 0         0    0                       13.0   
                                            1                       13.0   
                                            2                       13.0   
                                       1    0                       13.0   
                                            1                       13.0   
...                                                                  ...   
-274420.584639 272973.439868 12        52   1                       13.0   
                                            2                       13.0   
                                       53   0                       13.0   
                                            1                       13.0   
                                            2                       13.0   

                                                        Recovered      Slope  
x              y             treatment case size_class                        
-274600.584639 272853.439868 0         0    0            0.000000  13.703125  
                                            1            0.000000  13.703125  
                                            2            0.000000  13.703125  
                                       1    0            3.150391  13.703125  
                                            1            3.769531  13.703125  
...                                                           ...        ...  
-274420.584639 272973.439868 12        52   1            4.089844   7.453125  
                                            2            1.344727   7.453125  
                                       53   0            0.000000   7.453125  
                                            1            0.000000   7.453125  
                                            2            0.000000   7.453125  

[73710 rows x 5 columns]

In [ ]:
polygon_6.Recovered.sel(treatment = 1, case = 1)

<xarray.DataArray 'Recovered' (x: 7, y: 5, size_class: 3)>
dask.array<getitem, shape=(7, 5, 3), dtype=float16, chunksize=(7, 5, 3), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 -2.746e+05 -2.746e+05 ... -2.745e+05 -2.744e+05
  * y        (y) float64 2.729e+05 2.729e+05 2.729e+05 2.729e+05 2.73e+05
Dimensions without coordinates: size_class

In [ ]:
treatment_dict = {'Clearcut':0, '20_Thin_From_Above':1, '40_Thin_From_Above':2, '60_Thin_From_Above':3, '80_Thin_From_Above':4, '20_Thin_From_Below':5, '40_Thin_From_Below':6, '60_Thin_From_Below':7, '80_Thin_From_Below':8, '20_Proportional_Thin':9, '40_Proportional_Thin':10, '60_Proportional_Thin':11, '80_Proportional_Thin':12}

In [ ]:
def polygon_mask(polygon_ID, treatment, case, ds):
    ds = ds.rio.write_crs(crs)
    polygons = polygon_fetcher(polygon_ID)

    df = pd.DataFrame()

    for poly in polygons:
        x_min, y_min, x_max, y_max = poly.bounds
        box  = ds.sel(x =  slice(x_min - 30, x_max + 30), y = slice(y_min-30, y_max+30))
        masked = box.rio.clip([poly], all_touched = False) #origninal cbrec is false
        masked = masked.assign(specific_recovered = masked.Recovered.sel(treatment = treatment_dict[treatment], case = case))
        masked = masked[['Cell_To_Road','Nearest_Plant','Nearest_Plant_ID','specific_recovered','Slope']]
        df_temp = masked.to_dataframe()
        df_temp = df_temp.dropna(subset=['Slope'])
        df = pd.concat([df, df_temp])

    return df

In [ ]:
df_out = polygon_mask(6,'Clearcut', 1, polygon_6 )

In [ ]:
df_out.specific_recovered.unique()

array([0.9683, 0.957 , 0.331 , 2.248 , 2.203 , 0.5566, 1.132 , 1.194 ,
       0.2437, 1.102 , 0.9062, 0.298 , 1.236 , 1.28  , 0.3938, 2.63  ,
       2.87  , 0.837 ], dtype=float16)